In [2]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math

In [3]:
#import warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("../input/dmsp-particle-precipitation-aiready-data/DMSP_final_training.csv")
df.head()


# # Exploratory Data Analysis

In [5]:
df.info()

In [6]:
df.shape

In [7]:
#checking missing values
df.isnull().any().sum()

In [8]:
# statistical measures about the data
df.describe()

In [9]:
# checking the distribution of Target Varibale
df['ELE_TOTAL_ENERGY_FLUX'].value_counts()

In [10]:
object_columns = df.select_dtypes(include=['object']).columns

In [11]:
object_columns

In [12]:
# converting 'time' to datetime
df['Datetimes'] = pd.to_datetime(df['Datetimes'])

In [13]:
# extracting hour and minute from timestamp
df['hour'] = df['Datetimes'].dt.hour
df['minute'] = df['Datetimes'].dt.minute
df.drop('Datetimes', axis=1, inplace=True)

In [14]:
df.info()

# Building a Model

In [15]:
X = df.drop(columns='ELE_TOTAL_ENERGY_FLUX', axis=1)
Y = df['ELE_TOTAL_ENERGY_FLUX']

In [16]:
X

In [17]:
Y

In [18]:
### Splitting the data into training data & Testing data

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=2)

In [20]:
print(X.shape, X_train.shape, X_test.shape)

In [21]:
### Feature selection

In [22]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X_train, Y_train)
mutual_info

In [23]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

In [24]:
mutual_info.sort_values(ascending=False).plot.bar(figsize=(15,5))

In [25]:
from sklearn.feature_selection import SelectKBest

In [26]:
#No we Will select the  top 5 important features
sel_five_cols = SelectKBest(mutual_info_regression, k=5)
sel_five_cols.fit(X_train, Y_train)
X_train.columns[sel_five_cols.get_support()]

In [27]:
df['SC_AACGM_LAT'].value_counts

In [28]:
df['ID_SC'].value_counts

In [29]:
df['cos_SC_AACGM_LTIME'].value_counts

In [30]:
df['F107'].value_counts

In [31]:
df['vsw_6hr'].value_counts

In [32]:
X_train_new = sel_five_cols.transform(X_train)
X_test_new =  sel_five_cols.transform(X_test)

In [33]:
from keras.layers import BatchNormalization
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train_new.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='softmax'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='softmax'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='softmax'))
NN_model.add(BatchNormalization())

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

In [34]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [35]:
NN_model.fit(X_train_new,Y_train,epochs=100, validation_split=0.1,batch_size=128,callbacks=callbacks_list)

In [36]:
predictions = NN_model.predict(X_test_new)
predictions

In [37]:
# evaluate the keras model
_, accuracy = NN_model.evaluate(X_test_new, Y_test)
print('Accuracy: %.2f' % (accuracy*100))
_,r2_score = NN_model.evaluate(X_test_new, Y_test)
print("R2 score : %.2f" % r2_score)
_,mean_squared_error = NN_model.evaluate(X_test_new, Y_test)
print("mean_squared_error : %f" % mean_squared_error)      



In [43]:
#saving the model
filename = ('D:\DMSP Project\keras_NN_model.h5')
NN_model.save(filename)

In [44]:
#loading the model 
from keras.models import load_model
NN_model = load_model(filename)

In [45]:
# Load wights file of the best model :
wights_file = filename # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it\
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])